In [124]:
import pandas as pd
import numpy as np
import math

def format_with_uncertainty(value, error, unidad, max_error_digits=2):
    """
    Formatea un valor con su incertidumbre siguiendo las reglas de notación científica.
    
    Args:
        value (float): Valor medido
        error (float): Incertidumbre del valor
        max_error_digits (int): Número máximo de cifras significativas en el error
    
    Returns:
        str: Cadena formateada en notación científica
    """
    if pd.isna(value) or pd.isna(error) or error == 0:
        return f"{value:.2e}"
    
    # Encontrar el orden de magnitud del error
    error_order = math.floor(math.log10(abs(error)))
    
    # Calcular el factor de escala para redondear el error
    scale_factor = 10 ** (error_order - max_error_digits + 1)
    
    # Redondear el error al número deseado de cifras significativas
    rounded_error = round(error / scale_factor) * scale_factor
    
    # Determinar el orden de magnitud común (basado en el valor principal)
    if value != 0:
        value_order = math.floor(math.log10(abs(value)))
    else:
        value_order = error_order
    
    # Escalar ambos valores al mismo orden de magnitud
    common_scale = 10 ** value_order
    scaled_value = value / common_scale
    scaled_error = rounded_error / common_scale
    
    # Determinar el número de decimales necesarios
    if error_order >= value_order:
        decimals = max_error_digits - 1
    else:
        decimals = value_order - error_order + max_error_digits - 1
    
    decimals = max(0, decimals)  # No puede ser negativo
    
    # Formatear el resultado
    if value_order == 0:
        return f"({scaled_value:.{decimals}f} ± {scaled_error:.{decimals}f}) [{unidad}]"
    else:
        return f"({scaled_value:.{decimals}f} ± {scaled_error:.{decimals}f}) × 10^{value_order} [{unidad}]"

def format_dataframe_with_uncertainty(df, value_col, error_col, unidad, cifras_significativas = 2, result_col='formatted'):
    """
    Aplica el formateo con incertidumbre a un DataFrame completo.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos
        value_col (str): Nombre de la columna con los valores
        error_col (str): Nombre de la columna con las incertidumbres
        result_col (str): Nombre de la columna resultado
    
    Returns:
        pd.DataFrame: DataFrame con la columna formateada añadida
    """
    df_result = df.copy()
    df_result[result_col] = df.apply(
        lambda row: format_with_uncertainty(row[value_col], row[error_col], unidad, cifras_significativas), 
        axis=1
    )
    return df_result

def generar_archivo_cientifico(path, distancia_m = 1):
    tiempos_df = tiempos_df = pd.read_csv(path, delimiter= ';', dtype=float, usecols=[0])

    error_tiempo_s = 20e-6
    error_distancia_m = 1e-3 
    velocidad_sonido_m_s = distancia_m / tiempos_df
    error_velocidad_m_s = velocidad_sonido_m_s * np.sqrt((error_distancia_m/distancia_m)**2 + (error_tiempo_s/tiempos_df)**2)

    datos_df = pd.DataFrame()
    datos_df['tiempo'] = tiempos_df
    datos_df['distancia'] = distancia_m
    datos_df['error_distancia'] = error_distancia_m
    datos_df['error_tiempo'] = error_tiempo_s
    datos_df['velocidad'] = velocidad_sonido_m_s
    datos_df['error_velocidad'] = error_velocidad_m_s

    tiempo_formato_df = format_dataframe_with_uncertainty(datos_df, 'tiempo', 'error_tiempo', 's', cifras_significativas=1, result_col='tiempo_cientifico')
    distancia_formato_df = format_dataframe_with_uncertainty(tiempo_formato_df, 'distancia', 'error_distancia', 'm', cifras_significativas=1, result_col='distancia_cientifica')
    datos_formato_df = format_dataframe_with_uncertainty(distancia_formato_df, 'velocidad', 'error_velocidad', 'm/s', result_col='velocidad_cientifica')

    return datos_formato_df[['tiempo_cientifico', 'distancia_cientifica',  'velocidad_cientifica']]



In [125]:
velocidad_sonido_100cm_df = generar_archivo_cientifico(r'Datos\Tiempos 100cm.txt', distancia_m=2)
velocidad_sonido_80cm_df = generar_archivo_cientifico(r'Datos\Tiempos 80cm.txt', distancia_m=1.6)
velocidad_sonido_60cm_df = generar_archivo_cientifico(r'Datos\Tiempos 60cm.txt', distancia_m=1.2)
velocidad_sonido_40cm_df = generar_archivo_cientifico(r'Datos\Tiempos 40cm.txt', distancia_m=0.8)
velocidad_sonido_20cm_df = generar_archivo_cientifico(r'Datos\Tiempos 20cm.txt', distancia_m=0.4)

In [126]:
velocidad_sonido_100cm_df

,tiempo_cientifico,distancia_cientifica,velocidad_cientifica
0,(5.96 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.355 ± 0.011) × 10^2 [m/s]
1,(5.94 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.369 ± 0.011) × 10^2 [m/s]
2,(5.89 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.397 ± 0.012) × 10^2 [m/s]
3,(5.98 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.342 ± 0.011) × 10^2 [m/s]
4,(5.91 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.382 ± 0.012) × 10^2 [m/s]
5,(5.94 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.369 ± 0.011) × 10^2 [m/s]
6,(5.91 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.382 ± 0.012) × 10^2 [m/s]
7,(5.94 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.368 ± 0.011) × 10^2 [m/s]
8,(5.94 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.368 ± 0.011) × 10^2 [m/s]
9,(5.94 ± 0.02) × 10^-3 [s],(2.000 ± 0.001) [m],(3.368 ± 0.011) × 10^2 [m/s]


In [127]:
velocidad_sonido_80cm_df

,tiempo_cientifico,distancia_cientifica,velocidad_cientifica
0,(4.84 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.306 ± 0.014) × 10^2 [m/s]
1,(4.76 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.358 ± 0.014) × 10^2 [m/s]
2,(4.79 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.338 ± 0.014) × 10^2 [m/s]
3,(4.82 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.319 ± 0.014) × 10^2 [m/s]
4,(4.77 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.354 ± 0.014) × 10^2 [m/s]
5,(4.74 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.374 ± 0.014) × 10^2 [m/s]
6,(4.76 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.359 ± 0.014) × 10^2 [m/s]
7,(4.76 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.358 ± 0.014) × 10^2 [m/s]
8,(4.76 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.358 ± 0.014) × 10^2 [m/s]
9,(4.74 ± 0.02) × 10^-3 [s],(1.600 ± 0.001) [m],(3.374 ± 0.014) × 10^2 [m/s]


In [128]:
velocidad_sonido_60cm_df

,tiempo_cientifico,distancia_cientifica,velocidad_cientifica
0,(3.72 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.223 ± 0.018) × 10^2 [m/s]
1,(3.80 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.160 ± 0.017) × 10^2 [m/s]
2,(3.77 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.180 ± 0.017) × 10^2 [m/s]
3,(3.80 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.160 ± 0.017) × 10^2 [m/s]
4,(3.80 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.160 ± 0.017) × 10^2 [m/s]
5,(3.82 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.140 ± 0.017) × 10^2 [m/s]
6,(3.75 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.197 ± 0.017) × 10^2 [m/s]
7,(3.80 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.155 ± 0.017) × 10^2 [m/s]
8,(3.75 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.197 ± 0.017) × 10^2 [m/s]
9,(3.80 ± 0.02) × 10^-3 [s],(1.200 ± 0.001) [m],(3.155 ± 0.017) × 10^2 [m/s]


In [129]:
velocidad_sonido_40cm_df

,tiempo_cientifico,distancia_cientifica,velocidad_cientifica
0,(2.69 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(2.977 ± 0.022) × 10^2 [m/s]
1,(2.64 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.028 ± 0.023) × 10^2 [m/s]
2,(2.62 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.056 ± 0.024) × 10^2 [m/s]
3,(2.69 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(2.978 ± 0.022) × 10^2 [m/s]
4,(2.61 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.063 ± 0.024) × 10^2 [m/s]
5,(2.61 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.063 ± 0.024) × 10^2 [m/s]
6,(2.64 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.029 ± 0.023) × 10^2 [m/s]
7,(2.62 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.055 ± 0.024) × 10^2 [m/s]
8,(2.61 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.064 ± 0.024) × 10^2 [m/s]
9,(2.64 ± 0.02) × 10^-3 [s],(8.00 ± 0.01) × 10^-1 [m],(3.036 ± 0.023) × 10^2 [m/s]


In [130]:
velocidad_sonido_20cm_df

,tiempo_cientifico,distancia_cientifica,velocidad_cientifica
0,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
1,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
2,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
3,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
4,(1.15 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.466 ± 0.061) × 10^2 [m/s]
5,(1.18 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.393 ± 0.058) × 10^2 [m/s]
6,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
7,(1.17 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.410 ± 0.059) × 10^2 [m/s]
8,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
9,(1.16 ± 0.02) × 10^-3 [s],(4.00 ± 0.01) × 10^-1 [m],(3.463 ± 0.061) × 10^2 [m/s]
